# Large Scale Matrix Computations

In this notebookwe will walk through some of the more advanced things you can achieve with PyWren. Namely using S3 as a backing store we will implement a nearest neighbor classifier algorithm.

In [ ]:
%pylab inline
import boto3
import cloudpickle
import itertools
import concurrent.futures as fs
import io
import numpy as np
import time
from importlib import reload
from sklearn import metrics
import pywren
import pywren.wrenconfig as wc
import itertools
from operator import itemgetter
import sys
sys.path.insert(0,"..")
import matrix

In [ ]:
DEFAULT_BUCKET = wc.default()['s3']['bucket']
pwex = pywren.default_executor()

## 1. Matrix Multiplication

One nice thing about PyWren is it allows users to integrate existing python libraries easily.
For the following exercise, we are going to use some popular python libraries, e.g., NumPy, to work on some matrix multiplication problems.

In [ ]:
import numpy as np

def my_function(b):
    x = np.random.normal(0, b, 1024)
    A = np.random.normal(0, b, (1024, 1024))
    return np.dot(A, x)

res = pwex.map(my_function, np.linspace(0.1, 10, 100))
pywren.wait(res)
[f.result() for f in res]


## 2. Distributed Nearest Neighbor with Large Scale Matrix Multiplication 

A problem with the above method is that, we are limited to working with "small" matrices, that fit in the memory of a single lambda instance. With a little work we can write a "ShardedMatrix" wrapper that shards numpy matrices across S3 objects (the source code for this can be found in matrix.py) This allows us to use PyWren's map functionality to access different parts of the matrix. We can further use this functionality to compute a large scale matrix multiplication.


In the below example we will implement a distributed nearest neighbor implementation on top of PyWren and this ShardedMatrix abstraction. Note that nearest neighbor is often a hard to implement algorithm on BSP systems such as Apache Spark due to a high communication cost.

In [ ]:
from sklearn.datasets import fetch_mldata


Lets download the mnist dataset. This cell should take about 3 minutes to complete.

In [ ]:
X = fetch_mldata('MNIST original', data_home="/tmp/")['data'].astype('float32')
y = fetch_mldata('MNIST original', data_home="/tmp/")['target']
X_train = X[:60000, :]
y_train = y[:60000, np.newaxis]

X_test = X[60000:, :]
y_test = y[60000:, np.newaxis]

We will now "shard" the mnist matrix with a shard_size of 4000, what this means is that, we will convert the 60000 by 784 matrix into 30 separate 4000 x 784 numpy matrices that will be split across different S3 Keys. The first argument is the S3 folder where these submatrices can be found. This cell should take about 3 minutes to complete. 

In [ ]:
%time X_train_sharded = matrix.ShardedMatrix("x_train", shape=X_train.shape, bucket=DEFAULT_BUCKET, shard_sizes=[4000,784])
%time X_train_sharded.shard_matrix(X_train, n_jobs=16)

%time X_test_sharded = matrix.ShardedMatrix("x_test", shape=X_test.shape, bucket=DEFAULT_BUCKET, shard_sizes=[4000,784])
%time X_test_sharded.shard_matrix(X_test, n_jobs=16)

Now that we have our sharded matrices we can compute a local nearest neighbor classifier and compare it with one we will compute with PyWren. If we do everything correctly the PyWren implementation should be identical to the local one, but with a better scaling with dataset size.

In [ ]:
def compute_local_nearest_neighbor_labels(X_train, X_test, y_test, y_train):
    # compute a distance matrix
    train_norms = np.linalg.norm(X_train, axis=1)[:, np.newaxis] ** 2
    test_norms = np.linalg.norm(X_test, axis=1)[np.newaxis, :] ** 2
    return y_train[np.argmin(train_norms + -2*X_train.dot(X_test.T)+ test_norms, axis=0)]

In [ ]:
%time y_test_pred = compute_local_nearest_neighbor_labels(X_train, X_test, y_test, y_train)
print("Accuracy is ", metrics.accuracy_score(y_test_pred, y_test))

#### Lets try using PyWren.
Our strategy will be to use PyWren to map over (training point, testing point) pairs (in a blockwise fashion), and generate the distance matrix in a sharded form. Then we can launch another PyWren job to extract the nearest neighbors. 

In [ ]:
# create an "empty" ShardedMatrix on S3 (we will fill this Matrix in with pywren)
D_sharded = matrix.ShardedMatrix("D", shape=(X_train.shape[0], X_test.shape[0]), shard_sizes=[4000,4000], bucket=DEFAULT_BUCKET)

def compute_pywren_nearest_neighbor_distance_matrix(block_pair, X_train_sharded, X_test_sharded, D_sharded):
    block0,block1 = block_pair
    # compute a distance matrix block
    X_train_block = X_train_sharded.get_block(block0, 0)
    X_test_block = X_test_sharded.get_block(block1, 0)
    train_norms = np.linalg.norm(X_train_block, axis=1)[:, np.newaxis] ** 2
    test_norms = np.linalg.norm(X_test_block, axis=1)[np.newaxis, :] ** 2
    D_block = train_norms + -2*X_train_block.dot(X_test_block.T)+ test_norms
    D_sharded.put_block(block0, block1, D_block)
    return 0 

In [ ]:
pywren_distance_function = lambda bidx: compute_pywren_nearest_neighbor_distance_matrix(bidx, X_train_sharded, X_test_sharded, D_sharded)

In [ ]:
# check that our function works locally
pywren_distance_function((0,0)) # compute the first block of distance matrix
D_sharded.get_block(0,0)

In [ ]:
%time futures = pwex.map(pywren_distance_function, D_sharded.block_idxs)
%time pywren.wait(futures)
[f.result() for f in futures] # make sure nothing exceptioned

In [ ]:
def find_argmin(block_pair, D_sharded):
    D_block = D_sharded.get_block(*block_pair)
    offset = block_pair[0]*D_sharded.shard_sizes[0] 
    return (block_pair[1], offset + np.argmin(D_block, axis=0), np.min(D_block, axis=0))

In [ ]:
%time futures = pwex.map(lambda x: find_argmin(x, D_sharded), sorted(D_sharded.block_idxs))

In [ ]:
%time pywren.wait(futures)

In [ ]:
results = [f.result() for f in futures]

In [ ]:
def compute_results_from_pywren(results):
    mins = []
    for _, group in itertools.groupby(sorted(results, key=itemgetter(0)), key=itemgetter(0)):
        group = list(group)
        argmins = np.vstack([g[1] for g in group])
        argminmin = np.argmin(np.vstack([g[2] for g in group]), axis=0)
        mins.append(argmins[argminmin, np.arange(argmins.shape[1])])
    return np.hstack(mins)
y_test_pred_pywren = y_train[compute_results_from_pywren(results)]
print("Accuracy is ", metrics.accuracy_score(y_test, y_test_pred_pywren))



The advantage of this PyWren implementation is that we are executing in parallel over the entire matrix. So as we get more training and test points our implementation will scale gracefully along with it.

### Computing Training 1-NN

So one advantage of this PyWren implementation is that we can compute the training accuracy of our NN classifier.
Normally this is hard because the training distance matrix is 60000 x 60000. Which is around 10 GB and can be quite cumbersome to compute (and may not even fit in your local ram!).

(We know apriori for 1-NN it is always going to be 100% but we can verify that quickly with PyWren).

In [ ]:
# create an "empty" ShardedMatrix on S3 (we will fill this Matrix in with pywren)
D_train_sharded = matrix.ShardedMatrix("D", shape=(X_train.shape[0], X_train.shape[0]), shard_sizes=[4000,4000], bucket=DEFAULT_BUCKET)

In [ ]:
pywren_train_distance_function = lambda bidx: compute_pywren_nearest_neighbor_distance_matrix(bidx, X_train_sharded, X_train_sharded, D_train_sharded)
%time futures = pwex.map(pywren_train_distance_function, D_train_sharded.block_idxs)

In [ ]:
%time pywren.wait(futures)

In [ ]:
results = [f.result() for f in futures]

In [ ]:
%time futures = pwex.map(lambda x: find_argmin(x, D_train_sharded), sorted(D_train_sharded.block_idxs))

In [ ]:
%time pywren.wait(futures)

In [ ]:
results = [f.result() for f in futures]

In [ ]:
print("Training Accuracy is ", metrics.accuracy_score(y_train, y_train[compute_results_from_pywren(results)]))